In [ ]:
#!/usr/bin/env python

# coding: utf-8
 

#Webサイトのグループ化コード（クリニック領域だけとか、フォームだけとか、正規表現縛りができるとGAでの確認作業が楽になる

#配置もちゃんとできるようにする

 

#その他デジマ：問合せフォームと個人情報登録フォームのファネルを出せるようにする

In [ ]:
import pandas as pd
import numpy as np


# 新データ収集シート対応（ページビュー＝＞ユーザー）
df1 = pd.read_csv('D:/Users/fobayashi/Desktop/File/2022April4May20/PreviousPageNextPage_AllUU.csv')
PageViewFilter = 2 

df2 = pd.read_csv('D:/Users/fobayashi/Desktop/File/2022April4May20/ReferenceMediaPage_AllUU.csv')
df3 = pd.read_csv('D:/Users/fobayashi/Desktop/File/2022April4May20/Page_AllUU.csv')
df4 = pd.read_csv('D:/Users/fobayashi/Desktop/File/2022April4May20/ReferenceMedia_AllUU.csv')

# 前のページ遷移 -> ページ 
df1_noEnt=df1[df1["前のページ遷移"]!="(entrance)"]
df1_exerpt = df1_noEnt[df1_noEnt['ユーザー']>PageViewFilter]

# Lamda使って?以下を削除（?formId=があるものは残す）
df1_exerpt["前のページ遷移"] = pd.DataFrame(df1_exerpt["前のページ遷移"]).applymap(lambda x: x.split('?')[0] if (x.find('?') and not '?formId=' in x) else x)
df1_exerpt["ページ"] = pd.DataFrame(df1_exerpt["ページ"]).applymap(lambda x: x.split('?')[0] if (x.find('?') and not '?formId=' in x) else x)

# Lamda使って二つのドメインのみをスコープにする
df1_exerpt["前のページ遷移"] = pd.DataFrame(df1_exerpt["前のページ遷移"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x) else np.nan)
df1_exerpt["ページ"] = pd.DataFrame(df1_exerpt["ページ"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x) else np.nan)


#Nanになったいらない行を削除
df1_exerpt = df1_exerpt.dropna(how='any').reset_index().drop('index', axis=1)

#個人所法登録フォーム一元化
df1_exerpt["前のページ遷移"] = pd.DataFrame(df1_exerpt["前のページ遷移"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/MailForms" if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' and not x.split('/')[1] == 'form' else x)
df1_exerpt["前のページ遷移"] = pd.DataFrame(df1_exerpt["前のページ遷移"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/download-thanks" if x.split('/')[1] == 'download-thanks' else x)
df1_exerpt["前のページ遷移"] = pd.DataFrame(df1_exerpt["前のページ遷移"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/Forms" if x.split('/')[1] == 'form' else x)
df1_exerpt["ページ"] = pd.DataFrame(df1_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/MailForms" if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' and not x.split('/')[1] == 'form' else x)
df1_exerpt["ページ"] = pd.DataFrame(df1_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/download-thanks" if x.split('/')[1] == 'download-thanks' else x)
df1_exerpt["ページ"] = pd.DataFrame(df1_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/Forms" if x.split('/')[1] == 'form' else x)

# クリニック領域一元化
df1_exerpt["前のページ遷移"][df1_exerpt["前のページ遷移"].str.contains(www.olympus-medical.jp/gastroenterology/clinic, regex=True)] = 'gastroenterology/Clinic'
df1_exerpt["ページ"][df1_exerpt["ページ"].str.contains(www.olympus-medical.jp/gastroenterology/clinic, regex=True)] = 'gastroenterology/Clinic'


#['前のページ遷移', 'ページ']の組み合わせで重複したものを足し合わせ
df1_exerpt = df1_exerpt[['前のページ遷移', 'ページ', 'ユーザー']].groupby(['前のページ遷移', 'ページ'], as_index=False).sum()

df1_1 = df1_exerpt['前のページ遷移']
df1_2 = df1_exerpt['ページ']
df1_3 = df1_exerpt['ユーザー']

In [ ]:
# 参照元/メディア -> ページ

# df2 = pd.read_csv('C:/Users/fumi9/Desktop/File/ReferenceMedia_PageView.csv')
df2_exerpt = df2[df2['ユーザー']>PageViewFilter]

# Lamda使って?以下を削除（?formId=があるものは残す）
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: x.split('?')[0] if (x.find('?') and not '?formId=' in x) else x)
df2_exerpt["ページ"] = pd.DataFrame(df2_exerpt["ページ"]).applymap(lambda x: x.split('?')[0] if (x.find('?') and not '?formId=' in x) else x)

# Lamda使って二つのドメインのみをスコープにする
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x or ' / ' in x) and not 'olympusmedicalstg.prod.acquia-sites.com' in x else np.nan)
df2_exerpt["ページ"] = pd.DataFrame(df2_exerpt["ページ"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x or ' / ' in x) else np.nan)


# ()が悪さ？
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: x.replace(')', '').replace('(', '') if type(x) is str and ') / (' in x else x)


# スペースが悪さ？
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: x.replace(' ', '_') if type(x) is str and ' ' in x else x)


#Nanになったいらない行を削除
df2_exerpt = df2_exerpt.dropna(how='any').reset_index().drop('index', axis=1)


#個人所法登録フォーム一元化
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/MailForms" if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' and not x.split('/')[1] == 'form' else x)
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/download-thanks" if x.split('/')[1] == 'download-thanks' else x)
df2_exerpt["参照元/メディア"] = pd.DataFrame(df2_exerpt["参照元/メディア"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/Forms" if x.split('/')[1] == 'form' else x)
df2_exerpt["ページ"] = pd.DataFrame(df2_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/MailForms" if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' and not x.split('/')[1] == 'form' else x)
df2_exerpt["ページ"] = pd.DataFrame(df2_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/download-thanks" if x.split('/')[1] == 'download-thanks' else x)
df2_exerpt["ページ"] = pd.DataFrame(df2_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/Forms" if x.split('/')[1] == 'form' else x)

# クリニック領域一元化
df2_exerpt["ページ"][df2_exerpt["ページ"].str.contains(www.olympus-medical.jp/gastroenterology/clinic, regex=True)] = 'gastroenterology/Clinic'

#['前のページ遷移', 'ページ']の組み合わせで重複したものを足し合わせ
df2_exerpt = df2_exerpt[['参照元/メディア', 'ページ', 'ユーザー']].groupby(['参照元/メディア', 'ページ'], as_index=False).sum()

df2_1 = df2_exerpt['参照元/メディア']
df2_2 = df2_exerpt['ページ']
df2_3 = df2_exerpt['ユーザー']

df1_unique1 = df1_1.unique()
df1_unique2 = df1_2.unique()
df2_unique1 = df2_1.unique()
df2_unique2 = df2_2.unique()
df_unique = pd.Series(np.hstack((df1_unique1, df1_unique2, df2_unique1, df2_unique2))).unique()
df_unique = pd.DataFrame(df_unique).applymap(lambda x: x.split('/')[-2] + "/" + x.split('/')[-1] if '/' in x else x)


df_1= df2_1.append(df1_1).reset_index().drop('index', axis=1).to_numpy()
df_2= df2_2.append(df1_2).reset_index().drop('index', axis=1).to_numpy()
df_3= df2_3.append(df1_3).reset_index().drop('index', axis=1).to_numpy()


df_1 = pd.DataFrame(df_1).applymap(lambda x: x.split('/')[-2] + "/" + x.split('/')[-1] if '/' in x else x)
df_2 = pd.DataFrame(df_2).applymap(lambda x: x.split('/')[-2] + "/" + x.split('/')[-1] if '/' in x else x)

In [ ]:
# Position決めのためのWebページ種類分け 
groups = pd.DataFrame() 

# Main Groups
groups['media'] = df_unique[df_unique.applymap(lambda x: True if '_/_' in x else False)]
groups['top'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'www.olympus-medical.jp' else False)]
groups['category'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'gastroenterology' else False)]
groups['system'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'system' else False)]
groups['scope'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'scope' else False)]
groups['ai'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'ai' else False)]
groups['et'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'et' else False)]
groups['clinic'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'clinic' else False)]
groups['onlineseminar'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'onlineseminar' else False)]
groups['ma'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' else False)]
groups['maTh'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[1] == 'download-thanks' else False)]
groups['form'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'form' else False)]
 
# Others
groups['related'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'related' else False)]
groups['preview'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'preview' else False)]
groups['user'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'user' else False)]
groups['node'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'node' else False)]
groups['moderation'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'moderation' else False)]
groups['movie'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'movie' else False)]
groups['report'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'report' else False)]
groups['information'] = df_unique[df_unique.applymap(lambda x: True if x.split('/')[0] == 'information' else False)]

All_len = 0
for column_name, group in groups.iteritems():
    All_len = All_len + group.count()

print(len(df_unique), All_len, "equels?")

In [ ]:
# df_unique.to_csv('D:/Users/fobayashi/Desktop/File/2022April/xxx.csv')
# <span class="girk">Networkxによる描画部</span>

#円形配置：https://analytics-note.xyz/graph-theory/networkx-draw-pos/

poss = []

column_names = ['media',    'top', 'category', 'system', 'scope',  'ai',    'et',     'ma',   'clinic', 'onlineseminar', 'maTh', 'form', 'related',  'preview',  'user',    'node',  'moderation',  'movie',  'report', 'information']
offsets      = [[-20, 0],   [0, 1], [0, 0],   [0, 20],  [0, 10], [-10, 20], [0, -30], [20, 0], [0, -20],    [14, 0],   [25, 0],   [30,0], [20, 10],   [20, 7],   [20, 4],  [25, -10],   [20, -2],     [20, -5],   [20, -8],   [20, -10]]
Amps         = [[  7, 10],   [1, 1], [3, 4],   [3, 3],   [3, 3],   [1, 1],  [5, 5],   [3, 10], [3, 3],    [1, 1],       [1, 10], [1,10], [1, 1],    [1, 1],     [1, 1],   [1, 1],    [1, 1],      [1, 1],    [1, 1],    [1, 1]]

for i in range(0, len(column_names)):
    column_name = column_names[i]
    offset = offsets[i]
    Amp = Amps[i]
    pos = {
            n:  np.array([Amp[0]*np.cos(2*i*np.pi/groups[column_name].count())+offset[0], Amp[1]*np.sin(2*i*np.pi/groups[column_name].count())+offset[1]])
            for i, n in enumerate("['" + groups[column_name].dropna() + "']")
        }
    poss.append(pos)
PosFin = {}

for i in range(0, len(poss)):
    PosFin.update(poss[i])
# PosFin
# 全体UUよりノードの大きさ決め

In [ ]:
# df3_exerpt.to_csv("D:/Users/fobayashi/Desktop/File/2022April/1.csv")
# # pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: x.split('?')[0] if (x.find('?')) else x)
# df3_exerpt["ページ"]
# pd.DataFrame(df3_exerpt["ユーザー"]).applymap(lambda x: x if x=="" else x)
# df3_exerpt = df3
# pd.DataFrame(df3_exerpt["ユーザー"]).applymap(lambda x: x if type(x) is not float else 1)
# df3_exerpt[type(df3_exerpt['ユーザー'])==int]
# df3_exerpt['ページ']
 

In [ ]:
import pandas as pd
import numpy as np

# df3 = pd.read_csv('C:/Users/fumi9/Desktop/File/Page_AllUU.csv')
df3_exerpt = df3
df3_exerpt = df3_exerpt[df3_exerpt['ユーザー']>PageViewFilter]

#Lamda使って?以下を削除（?formId=があるものは残す）
# df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: x if type(x) is str else np.nan)
df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: x.split('?')[0] if (type(x) is str and x.find('?') and not '?formId=' in x) else x)
# df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: x.split('・・')[0] if (type(x) is str and x.find('・・')) else x)
# df3_exerpt["ユーザー"] = pd.DataFrame(df3_exerpt["ユーザー"]).applymap(lambda x: x if type(x) is not float else 1)


# Lamda使って二つのドメインのみをスコープにする
df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: x if (type(x) is str and 'www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x) else np.nan)
 

#Nanになったいらない行を削除
df3_exerpt = df3_exerpt.dropna(how='any').reset_index().drop('index', axis=1)


# 個人所法登録フォーム一元化
df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/MailForms" if x.split('/')[0] == 'oj-iryo2.olympus-medical.jp' and not x.split('/')[1] == 'download-thanks' and not x.split('/')[1] == 'form' else x)
df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/download-thanks" if x.split('/')[1] == 'download-thanks' else x)
df3_exerpt["ページ"] = pd.DataFrame(df3_exerpt["ページ"]).applymap(lambda x: "oj-iryo2.olympus-medical.jp/Forms" if x.split('/')[1] == 'form' else x)

# クリニック領域一元化
df3_exerpt["ページ"][df3_exerpt["ページ"].str.contains(www.olympus-medical.jp/gastroenterology/clinic, regex=True)] = 'gastroenterology/Clinic'

#['前のページ遷移', 'ページ']の組み合わせで重複したものを足し合わせ
df3_exerpt = df3_exerpt[['ページ', 'ユーザー']].groupby(['ページ'], as_index=False).sum()

df3_1 = df3_exerpt['ページ']
df3_2 = df3_exerpt['ユーザー']
df3_1_1 = pd.DataFrame(df3_1).applymap(lambda x: x.split('/')[-2] + "/" + x.split('/')[-1] if '/' in x else x)['ページ']

#並べ替え
Newdf3 = pd.DataFrame([df3_1_1,df3_2]).T

import pandas as pd
import numpy as np

# df4 = pd.read_csv('C:/Users/fumi9/Desktop/File/ReferenceMedia_AllUU.csv')
df4_exerpt = df4


# # Lamda使って?以下を削除（?formId=があるものは残す）
# df4_exerpt["参照元/メディア"] = pd.DataFrame(df4_exerpt["参照元/メディア"]).applymap(lambda x: x.split('?')[0] if (type(x) is str and x.find('?') and not '?formId=' in x) else x)
# df4_exerpt["ページ"] = pd.DataFrame(df4_exerpt["ページ"]).applymap(lambda x: x.split('?')[0] if (x.find('?') and not '?formId=' in x) else x)
# # Lamda使って二つのドメインのみをスコープにする
# df4_exerpt["参照元/メディア"] = pd.DataFrame(df4_exerpt["参照元/メディア"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x or ' / ' in x) and not 'olympusmedicalstg.prod.acquia-sites.com' in x else np.nan)
# df4_exerptdf4_exerptdf4_exerpt["ページ"] = pd.Datdf4_exerptaFrame(df2_exerpt["ページ"]).applymap(lambda x: x if type(x) is str and ('www.olympus-medical.jp' in x or 'oj-iryo2.olympus-medical.jp' in x or ' / ' in x) else np.nan)

# ()が悪さ？
df4_exerpt["参照元/メディア"] = pd.DataFrame(df4_exerpt["参照元/メディア"]).applymap(lambda x: x.replace(')', '').replace('(', '') if type(x) is str and ') / (' in x else x)

# スペースが悪さ？
df4_exerpt["参照元/メディア"] = pd.DataFrame(df4_exerpt["参照元/メディア"]).applymap(lambda x: x.replace(' ', '_') if type(x) is str and ' ' in x else x)

#Nanになったいらない行を削除
df4_exerpt = df4_exerpt.dropna(how='any').reset_index().drop('index', axis=1)

#['前のページ遷移', 'ページ']の組み合わせで重複したものを足し合わせ
df4_exerpt = df4_exerpt[['参照元/メディア', 'ユーザー']].groupby(['参照元/メディア'], as_index=False).sum()
df4_1 = df4_exerpt['参照元/メディア']
df4_2 = df4_exerpt['ユーザー']
df4_ = pd.DataFrame([df4_1, df4_2]).T.set_axis(['ページ', 'ユーザー'], axis=1)

# 参照元/メディアとページを結合
Newdf34 = Newdf3.append(df4_).reset_index(drop=True)
df_unique_dummy = df_unique.copy()
df_unique_copy = df_unique.copy()

In [ ]:
# networkxでグラフ
get_ipython().run_line_magic('matplotlib', 'inline')

# ↑jupter notebookで使用する場合のマジックコマンド
import matplotlib.pyplot as plt
import networkx as nx
import plotly
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

G = nx.DiGraph()
for node in df_unique:
    G.add_node(str(node))
G.remove_node('0') #なぜか表示されていたので削除
LineWeight = [n for n in df_3]

for i in range(0, len(df_1)):
#     print(str(df_1.to_numpy().tolist()[i]), str(df_2.to_numpy().tolist()[i]), LineWeight[i][0])
    G.add_edge(str(df_1.to_numpy().tolist()[i]), str(df_2.to_numpy().tolist()[i]), weight=LineWeight[i][0]) #label='1')
# dict(G.degree()) 

tmp = pd.DataFrame(G.nodes()).applymap(lambda x: x.replace('[', '').replace(']', '').replace("'", "") if type(x) is str and '[' in x else x)

# 遷移数に存在するリンクの全体UUをnodesの並びと同じにする
df_unique_copy = tmp

for i in range(0, len(df_unique_copy)):
    df_unique_dummy.iloc[i,0] = Newdf34[Newdf34['ページ'] == df_unique_copy.iloc[i,0]]['ユーザー'].sum()

fig = plt.figure(figsize=(12, 10))
# pos = nx.spring_layout(G, k=100, iterations=3, seed=3)
# pos = nx.circular_layout(G)
pos = PosFin

# for node in G.nodes():
#     G.nodes[node]["pos"] = pos[node]

# widthをGの並び替え
# dfNew: ユーザー数アリ、dfG: G.edgesの並び
dfNew = pd.concat([df_1[0], df_2[0], pd.DataFrame(df_3)[0]], axis=1).set_axis(['前ページ', 'ページ', 'ユーザー'], axis=1)
dfG = pd.DataFrame(columns=['前ページ', 'ページ'])

i=0
for edge in G.edges():
    dfG = dfG.append(pd.DataFrame({(edge[0], edge[1])}, columns=['前ページ', 'ページ']),ignore_index=True)
    i=i+1
dfG = dfG.applymap(lambda x: x.replace('[', '').replace(']', '').replace("'", "") if type(x) is str and '[' in x else x)

dfNew['前後ページ'] = pd.DataFrame(dfNew['前ページ'] + '&' + dfNew['ページ'])
dfG['前後ページ'] = pd.DataFrame(dfG['前ページ'] + '&' + dfG['ページ'])


dfNew = dfNew.drop('前ページ', axis=1).drop('ページ', axis=1)
dfG = dfG.drop('前ページ', axis=1).drop('ページ', axis=1)
dfLineWidth_dummy = dfNew.copy()

for i in range(0, len(df_unique_copy)):
    dfLineWidth_dummy.iloc[i,0] = dfNew[dfNew['前後ページ'] == dfG.iloc[i,0]]['ユーザー']#.sum()


# LineSize networkxで表示するとなんか変、Gephiはあってる
LineSize = [np.array([n/200]) for n in dfLineWidth_dummy['ユーザー']]
nx.draw_networkx_edges(G, pos, alpha=0.6, edge_color='g',  width=LineSize)

df34ForColor = df_unique_copy.copy()
df34ForColor.columns=['ページ']
df34ForColor['color'] = 'blue'
for i in range(0, len(df34ForColor['ページ'])):
    if '_/_' in df34ForColor['ページ'][i]:
        df34ForColor['color'][i] = 'yellow'
    elif 'oj-iryo2' in df34ForColor['ページ'][i]:
        df34ForColor['color'][i] = 'black'
    else:
        df34ForColor['color'][i] = 'blue'
# nx.draw_networkx_nodes(G, pos, node_color='b', alpha=0.5)
nx.draw_networkx_nodes(G, pos, node_color=df34ForColor['color'], alpha=0.5, node_size=df_unique_dummy.to_numpy().tolist()) #size指定


# ラベルつけるかどうか
nx.draw_networkx_labels(G, pos, font_size=8, alpha=1)
plt.show()
fig.savefig("img.png")

# 出力
sizes = df_unique_dummy.to_numpy().tolist()
poss = PosFin

df34ForColor['rgb']=df34ForColor['color'].copy()
for i in range(0, len(df34ForColor['rgb'])):
    if df34ForColor['color'][i] == 'blue':
        df34ForColor['rgb'][i]={'r': 0, 'g': 0, 'b': 255, 'a': 1}
    elif df34ForColor['color'][i] == 'yellow':
        df34ForColor['rgb'][i]={'r': 255, 'g': 255, 'b': 0, 'a': 1}
    elif df34ForColor['color'][i] == 'black':
        df34ForColor['rgb'][i]={'r': 0, 'g': 0, 'b': 0, 'a': 1}
    else:
        df34ForColor['rgb'][i]={'r': 128, 'g': 0, 'b': 128, 'a': 1}

i=0
for g in G.nodes():
    G.nodes[g]['viz'] = {'size': sizes[i][0],
#                         'color': {'r': color[g][0], 'g': color[g][1], 'b': color[g][2], 'a': 1},
#                         'color': {'r': 0, 'g': 0, 'b': 255, 'a': 0.5},
                         'color': df34ForColor['rgb'][i],
                        'position': {'x': poss[g][0]*1000, 'y': poss[g][1]*1000, 'z': 0}}
    i=i+1

for g in G.edges():
    G.edges[g]['viz'] = {'color': { 'r': 0, 'b': 0, 'g': 255, 'a': 0.6}}

#あとノード色変えたい
nx.write_gexf(G, "./test2.gexf", version = '1.2draft') 

# 出力したgexfファイルのLabelにAllUUの数値を追加
import xml.etree.ElementTree as ET

tree = ET.parse('./test2.gexf')
# tree = ET.parse('./sports.xml')

# XMLを取得
root = tree.getroot()

# AllUUのこと
sizesForXml = df_unique_dummy.to_numpy().tolist()

i=0
for child in root[1][0]: # nodes
    child.attrib['label'] = child.attrib['label'] + ":" + str(sizesForXml[i][0])
    i=i+1

# XMLファイル書き込み
tree.write('./test3_draw.gexf', encoding='UTF-8')
 